This notebook aims to check the suitable TV filter for hyperspectral data.

We first try to reproduce Fig. 14 of this [paper](https://doi.org/10.5194/egusphere-2023-1962) and then calculate our own TV filter weights.

## Synthetic Noisy Data

In [1]:
import numpy as np
from skimage.restoration import denoise_tv_chambolle
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt
import proplot as pplt

pplt.rc['figure.facecolor'] = 'white'

The precision of hyperspectral imageries are from 3% to 7% ([Cusworth et al. (2019)](https://doi.org/10.5194/amt-12-5655-2019)):

<img src="https://amt.copernicus.org/articles/12/5655/2019/amt-12-5655-2019-f05-web.png" alt="drawing" width="500"/>

In [2]:
# Background methane
bg_ch4_ppb = 1850 # ppb

# Image size
n_imgs = 300
img_size = (20, 20)

# Measurement precision
precision = [0.03, 0.04, 0.05, 0.06] 

# Sweep parameters
lambdas = np.arange(5, 250, 5)
min_pix_ths = np.arange(50, 800, 50)

# Loop through noise levels
noisy_ch4 = []
for noise_level in precision:

    # Add background 
    ch4_img = np.ones((n_imgs, *img_size)) * bg_ch4_ppb
    
    # Add noise
    noise = np.random.normal(loc=0, scale=noise_level*bg_ch4_ppb, size=(n_imgs, *img_size))
    noisy_ch4_img = ch4_img + noise
    
    noisy_ch4.append(noisy_ch4_img)

XCH4_noisy = np.stack(noisy_ch4, axis=0)

In [3]:
XCH4_noisy.shape

(4, 300, 20, 20)

In [4]:
# Functions
def get_plume_masks(img, min_pix, threshold):
    mask = img > threshold
    labels = label(mask)
    regions = regionprops(labels)
    valid_labels = [r.label for r in regions if r.area >= min_pix]
    return np.isin(labels, valid_labels)

# apply TV filter
def smooth(data, weight):
    return denoise_tv_chambolle(data, weight=weight)

def calc_detection(id_precision):
    # Store results
    min_pix_lims = []
    thresholds = []

    # Loop through lambdas
    for lam in lambdas:

        # Denoise
        denoised = denoise_tv_chambolle(XCH4_noisy[id_precision, :, :], weight=lam)

        # Get threshold
        thresh = 2*np.std(denoised) + bg_ch4_ppb
        thresholds.append(thresh)

        # Get false detections
        falsely_detected_mass = []
        for min_pix in min_pix_ths:
            masks = get_plume_masks(denoised, min_pix, thresh)
            mass = np.sum(masks)
            falsely_detected_mass.append((lam, min_pix, mass))

        # Get min pixel threshold
        # vals = [v for v in falsely_detected_mass if v[0]==lam]
        try:
            min_pix = min([v[1] for v in falsely_detected_mass if v[2]==0])
            min_pix_lims.append((lam, min_pix))
        except ValueError:
            pass

    return min_pix_lims, thresholds

In [5]:
results = {}

for index in range(len(precision)):
    print(f'Processing precision: {precision[index]}')
    min_pix_lims, thresholds = calc_detection(index)
    results[precision[index]] = [min_pix_lims, thresholds]

Processing precision: 0.03
Processing precision: 0.04
Processing precision: 0.05
Processing precision: 0.06


In [ ]:
fig, axs = pplt.subplots(nrows=2, ncols=2, share=2, span=False, ycolor='blue6')

for idx in range(len(results)):
    ax = axs[idx]
    ax.plot([v[0] for v in results[precision[idx]][0]], [v[1] for v in results[precision[idx]][0]], color='blue6', label='min pixel')

    if idx %2 != 0:
        ax2 = ax.alty(color='orange6', label='$\Delta$XCH$_4$ threshold')
    else:
        ax2 = ax.alty(color='orange6')

    ax2.plot(lambdas, results[precision[idx]][1], color='orange6', label='XCH4 threshold')

    ax.format(title=f'Precision: {precision[idx]*100}%')

axs.format(xlabel='Smoothing Weight', ylabel='Min Pixel Threshold', grid=False)

# fig.savefig('tv_filter_param.pdf')

> At low smoothing levels (λ < 45) XCH4 detection threshold decreases sharply with only small increases in nmin. Beyond this the pixel threshold limit continues to increase with little improvement in the XCH4 threshold (Christopher et al. 2023)

For our cases, it is suitable to set the default weight as 50.

## Real Data

In [10]:
from hyperch4 import Hyper
from glob import glob

In [11]:
data_dir = '/Users/xinz/Documents/github/HyperCH4/test_data/'
filename = glob(data_dir+'ENMAP01-____L1B-DT0000015090_20230424T060259Z_001_V010201_20230426T035016Z.ZIP')
hyp = Hyper(filename, reader='hsi_l1b') # "hsi_l1b", "emit_l1b", "hyc_l1"

hyp.load()

/Users/xinz/Documents/github/satpy_hyper/satpy/readers/hsi_l1b.py:53: RuntimeWarning: No digital elevation model provided. Note that this may cause uncertainties, e.g., in the atmospheric correction and the orthorectification.
  self.cfg = EnPTConfig(**config_minimal)


In [12]:
hyp.retrieve(wvl_intervals=[1300, 2500])
ch4_swir = hyp.scene['ch4'].isel(bands=0)
hyp.retrieve(wvl_intervals=[2100, 2450])
ch4 = hyp.scene['ch4'].isel(bands=0)

In [13]:
diff = ch4_swir - ch4
scale = ch4.std()/ch4_swir.std()
# scale if ch4_swir < ch4
hyp.scene['ch4_comb'] = ch4.where(diff>0, ch4_swir*scale).rename('ch4_comb')

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'axes.titlesize': 'xx-large'})

denoised_ch4 = denoise_tv_chambolle(ch4.data, weight=50)
denoised_ch4_swir = denoise_tv_chambolle(ch4_swir.data, weight=50)

fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(16, 8))
axs = axs.flatten()

ax = axs[0]
m = ax.imshow(ch4, vmin=0, vmax=200, cmap='viridis')
# plt.colorbar(m)
ax.set_title('default imshow (2100-2450 nm)')

ax = axs[1]
m = ax.imshow(ch4, vmin=0, vmax=200, interpolation='none', cmap='viridis')
# plt.colorbar(m)
ax.set_title('imshow without interp (2100-2450 nm)')

ax = axs[2]
m = ax.imshow(denoised_ch4, vmin=0, vmax=200, interpolation='none', cmap='viridis')
cb = plt.colorbar(m)
cb.ax.set_ylabel('$\Delta$CH$_4$ (ppb)')
ax.set_title('TV filter (2100-2450 nm)')

ax = axs[3]
m = ax.imshow(ch4_swir, vmin=0, vmax=200, cmap='viridis')
# plt.colorbar(m)
ax.set_title('default imshow (1300-2500 nm)')

ax = axs[4]
m = ax.imshow(ch4_swir, vmin=0, vmax=200, interpolation='none', cmap='viridis')
# plt.colorbar(m)
ax.set_title('imshow without interp (1300-2500 nm)')

ax = axs[5]
m = ax.imshow(denoised_ch4_swir, vmin=0, vmax=200, interpolation='none', cmap='viridis')
cb = plt.colorbar(m)
cb.ax.set_ylabel('$\Delta$CH$_4$ (ppb)')
ax.set_title('TV filter (1300-2500 nm)')